# Multiple Period Production-Inventory Model

## Problem Statement
Acme Manufacturing Company has a contract to deliver 100, 250, 190, 140, 220, and 110 home windows over the next 6 months. Production cost (labor, material, and utilities) per window varies by period and is estimated to be \\$50, \\$45, \\$55, \\$48, \\$52, and \\$50 over the next 6 months.
To take advantage of the fluctuations in manufacturing cost, Acme can produce more windows than needed in a given month and hold the extra units for delivery in later months. This will incur a storage cost at the rate of $8 per window per month, assessed on end-of-month inventory. Develop a linear program to determine the optimum production schedule.

This problem is taken from Example 2.4-3 from the book **Operations Research: An Introduction, 10th ed.** (Taha, 2017).

## Mathematical Model
The variables of the problem include the monthly production amount and the end-of-month inventory. For $i=1,2,...,6$, let
$$x_i=\text{Number of units produced in month } i$$
$$I_i=\text{Inventory units left at the end of month } i$$

The system starts empty $(I_0 = 0)$.

$$\text{Total production cost }= 50x_i + 45x_2 + 55x_3 + 48x_4 +52x_5 + 50x_6$$

$$\text{Total inventory (storage) cost}= 8(I_1 + I_2 + I_3 + I_4 + I_5 + I_6  $$

Thus the objective function is

$$\text{Minimize } z=50x_i + 45x_2 + 55x_3 + 48x_4 +52x_5 + 50x_6 + 8(I_1 + I_2 + I_3 + I_4 + I_5 + I_6)$$

## Constraints
The constraints for the problem are mathematically translated for the individual month as:

$$x_1 - I_1 = 100 \quad \text{(Month 1)}$$
$$I_1 + x_2 - I_2 = 250 \quad \text{(Month 1)}$$
$$I_2 +x_3 - I_3 = 190 \quad \text{(Month 1)}$$
$$I_3 +x_4 - I_4 = 140 \quad \text{(Month 1)}$$
$$I_4 +x_5 - I_5 = 220 \quad \text{(Month 1)}$$
$$I_5 +x_6 = 110 \quad \text{(Month 1)}$$

## Linear Programming using PuLP
We can solve this optimization problem using the PuLP library.

In [46]:
# Importing the necessary libraries/packages
from pulp import *

In [47]:
# Defining the problem data
model = LpProblem(name="prod-inv-problem", sense=1)

In [48]:
# Defining the decision variables (cannot be less than zero and is an integer)
x1 = LpVariable(name="month1", lowBound=0, cat="LpInteger")
x2 = LpVariable(name="month2", lowBound=0, cat="LpInteger")
x3 = LpVariable(name="month3", lowBound=0, cat="LpInteger")
x4 = LpVariable(name="month4", lowBound=0, cat="LpInteger")
x5 = LpVariable(name="month5", lowBound=0, cat="LpInteger")
x6 = LpVariable(name="month6", lowBound=0, cat="LpInteger")

i1 = LpVariable(name="inv1", lowBound=0, cat="LpInteger")
i2 = LpVariable(name="inv2", lowBound=0, cat="LpInteger")
i3 = LpVariable(name="inv3", lowBound=0, cat="LpInteger")
i4 = LpVariable(name="inv4", lowBound=0, cat="LpInteger")
i5 = LpVariable(name="inv5", lowBound=0, cat="LpInteger")
i6 = LpVariable(name="inv6", lowBound=0, cat="LpInteger")

In [49]:
# Constructing the objective function
model += 50*x1 + 45*x2 + 55*x3 + 48*x4 + 52*x5 + 50*x6 + 8*(i1 + i2 + i3 + i4 + i5 + i6)

In [50]:
# Defining the constraints
model += x1 - i1 >= 100, "month1_req"
model += i1 + x2 - i2 >= 250, "month2_req"
model += i2 + x3 - i3 >= 190, "month3_req"
model += i3 + x4 - i4 >= 140, "month4_req"
model += i4 + x5 - i5 >= 220, "month5_req"
model += i5 + x6 >= 110, "month6_req"

In [51]:
# Solving the objective function
model.solve()
print("Status:", LpStatus[model.status])

Status: Optimal


In [52]:
for v in model.variables():
    print(v.name, "=", v.varValue)

inv1 = 0.0
inv2 = 190.0
inv3 = 0.0
inv4 = 0.0
inv5 = 0.0
inv6 = 0.0
month1 = 100.0
month2 = 440.0
month3 = 0.0
month4 = 140.0
month5 = 220.0
month6 = 110.0


In [53]:
print("The optimized cost = ", value(model.objective))

The optimized cost =  49980.0


## Results of PuLP

The optimization found the optimized cost to be $49980.0 achieved using the following optimum production schedule:

| Month | Production | End-of-Month Inventory |
| --- | --- | --- |
| 1 | 100 | 0 |
| 2 | 440 | 190 |
| 3 | 0 | 0 |
| 4 | 140 | 0 |
| 5 | 220 | 0 |
| 6 | 110 | 0 |

## Linear Programming using SciPy
Using the same problem, we can also use the SciPy library to solve optimization problems.

In [54]:
#Import libraries
import numpy as np
from scipy.optimize import linprog

In [55]:
# Coefficients of the objective function
c = [50, 45, 55, 48, 52, 50, 8, 8, 8, 8, 8, 8]

In [56]:
# CONSTRAINTS

# Left-hand side of the inequalities
A = [[-1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
     [0, -1, 0, 0, 0, 0, -1, 1, 0, 0, 0, 0],
     [0, 0, -1, 0, 0, 0, 0, -1, 1, 0, 0, 0],
     [0, 0, 0, -1, 0, 0, 0, 0, -1, 1, 0, 0],
     [0, 0, 0, 0, -1, 0, 0, 0, 0, -1, 1, 0],
     [0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0]]

# Right-hand side of the inequalities
b = [-100, -250, -190, -140, -220, -110]

In [57]:
# bounds
bounds = [(0, None)] * 12

In [58]:
res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method="highs")

# Printing the results
print(f"Status: {res.message}")
print(f"Production amounts:", res.x[:6])
print(f"Inventory balances:", res.x[6:])
print(f"Minimum cost: {res.fun}")

Status: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Production amounts: [100. 440.   0. 140. 220. 110.]
Inventory balances: [  0. 190.   0.   0.   0.   0.]
Minimum cost: 49980.0


## Results of SciPy
The optimization using the SciPy library found the same exact solution as the PuLP. The optimized cost for the production-inventory problem is $49980.0 using the production schedule:

| Month | Production | End-of-Month Inventory |
| --- | --- | --- |
| 1 | 100 | 0 |
| 2 | 440 | 190 |
| 3 | 0 | 0 |
| 4 | 140 | 0 |
| 5 | 220 | 0 |
| 6 | 110 | 0 |

## References
- Taha, H. (2017). Operations Research: An Introduction (10th ed.). London: Pearson.
- Optimization with PuLP — PuLP 3.3.0 documentation. (2026). Github.io. https://coin-or.github.io/pulp/index.html
- SciPy Optimize Linprog (2026). https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html
- salinmunlu47 (2020). SciPy Optimization Examples. https://github.com/salimunlu47/scipy-optimization-examples